In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.6.0'

In [11]:
# Create a constant op
# This op is added as a node to the default graph
hello = tf.constant("Hello, Tensorflow!")
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)

# run the op and get result
tf.print(hello, node1, node2)

"Hello, Tensorflow!" 3 4


In [10]:
node3 = tf.add(node1, node2)

In [12]:
print("node1: ", node1, "node2: ", node2)
print("node3: ", node3)

node1:  tf.Tensor(3.0, shape=(), dtype=float32) node2:  tf.Tensor(4.0, shape=(), dtype=float32)
node3:  tf.Tensor(7.0, shape=(), dtype=float32)


In [7]:
# tensorflow v1.0대에서는 Session()으로 세션을 만들고 run()을 해야했지만, v2.0부터는 이 과정이 생략된다. v1은 아래처럼 실행한다.
with tf.compat.v1.Session() as sess:
  # Build a graph. 이 방식으로 사용하려면 세션을 만들면서 그래프를 빌드해야 한다.
  node1 = tf.constant(3.0, dtype=tf.float32)
  node2 = tf.constant(4.0)
  print("sess.run(node1, node2): ", sess.run([node1, node2]))

sess.run(node1, node2):  [3.0, 4.0]


In [8]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)
tf.print([node1, node2])

[3, 4]


In [9]:
@tf.function
def adder_node(a, b):
  return a + b

tf.print(adder_node(3, 4))

7


## Build hypothesis and cost


In [24]:
x_data = [1, 2, 3, 4, 5]
y_data = [1, 2, 3, 4, 5]

W = tf.Variable(2.9)
b = tf.Variable(0.5)

hypothesis = W * x_data + b

In [25]:
cost = tf.reduce_mean(tf.square(hypothesis - y_data))

In [26]:
# tf.reduce_mean()
v = [1., 2., 3., 4.]
print(tf.reduce_mean(v))

# tf.square()
print(tf.square(3))

print(hypothesis - y_data)

tf.Tensor(2.5, shape=(), dtype=float32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor([ 2.4       4.3       6.200001  8.1      10.      ], shape=(5,), dtype=float32)


## Gradient descent

In [27]:
# 학습률, 얼마만큼 grad 값을 반영할 것인지
learning_rate = 0.01

for i in range(200):
  with tf.GradientTape() as tape:
    hypothesis = W * x_data + b
    cost = tf.reduce_mean(tf.square(hypothesis - y_data))

  # gradient 메서드를 통해 경사도값, 즉 미분값을 구한다. 첫 번째 인자(cost)인 함수에 대해서 두 번째 인자 리스트의 변수 들에 대해 개별 미분 값을 반환
  W_grad, b_grad = tape.gradient(cost, [W, b])

  # W와 b값을 업데이트: assign_sub는 -=연산자 역할을 한다. 즉, W -= learning_rate * W_grad
  W.assign_sub(learning_rate * W_grad)
  b.assign_sub(learning_rate * b_grad)

  # with ~ 여기까지가 한 걸음
  if i % 10 == 0:
    print("{:5}|{:10.4}|{:10.4}|{:10.6f}".format(i, W.numpy(), b.numpy(), cost))

    0|     2.452|     0.376| 45.660004
   10|     1.104|  0.003398|  0.206336
   20|     1.013|  -0.02091|  0.001026
   30|     1.007|  -0.02184|  0.000093
   40|     1.006|  -0.02123|  0.000083
   50|     1.006|  -0.02053|  0.000077
   60|     1.005|  -0.01984|  0.000072
   70|     1.005|  -0.01918|  0.000067
   80|     1.005|  -0.01854|  0.000063
   90|     1.005|  -0.01793|  0.000059
  100|     1.005|  -0.01733|  0.000055
  110|     1.005|  -0.01675|  0.000051
  120|     1.004|  -0.01619|  0.000048
  130|     1.004|  -0.01566|  0.000045
  140|     1.004|  -0.01513|  0.000042
  150|     1.004|  -0.01463|  0.000039
  160|     1.004|  -0.01414|  0.000037
  170|     1.004|  -0.01367|  0.000034
  180|     1.004|  -0.01322|  0.000032
  190|     1.004|  -0.01278|  0.000030


In [30]:
# predict
print(W * 5 + b)
print(W * 2.5 + b)

tf.Tensor(5.0047708, shape=(), dtype=float32)
tf.Tensor(2.4961886, shape=(), dtype=float32)


## Revised Tensorflow Code
>Simplified cost function

In [31]:
# Python code
import numpy as np

X = np.array([1, 2, 3])
Y = np.array([1, 2, 3])

def cost_func(W, X, Y):
  c = 0
  for i in range(len(X)):
    c += (W * X[i] - Y[i]) ** 2
  return c / len(X)

for feed_W in np.linspace(-3, 5, num=15):
  curr_cost = cost_func(feed_W, X, Y)
  print("{:6.3f} | {:10.5f}".format(feed_W, curr_cost))

-3.000 |   74.66667
-2.429 |   54.85714
-1.857 |   38.09524
-1.286 |   24.38095
-0.714 |   13.71429
-0.143 |    6.09524
 0.429 |    1.52381
 1.000 |    0.00000
 1.571 |    1.52381
 2.143 |    6.09524
 2.714 |   13.71429
 3.286 |   24.38095
 3.857 |   38.09524
 4.429 |   54.85714
 5.000 |   74.66667


In [33]:
# TensorFlow code
X = np.array([1, 2, 3])
Y = np.array([1, 2, 3])

def cost_func(W, X, Y):
  hypothesis = W * X
  return tf.reduce_mean(tf.square(hypothesis - Y))

W_values = np.linspace(-3, 5, num=15)
cost_values = []

for feed_W in W_values:
  curr_cost = cost_func(feed_W, X, Y)
  cost_values.append(curr_cost)
  print("{:6.3f} | {:10.5f}".format(feed_W, curr_cost))

-3.000 |   74.66667
-2.429 |   54.85714
-1.857 |   38.09524
-1.286 |   24.38095
-0.714 |   13.71429
-0.143 |    6.09524
 0.429 |    1.52381
 1.000 |    0.00000
 1.571 |    1.52381
 2.143 |    6.09524
 2.714 |   13.71429
 3.286 |   24.38095
 3.857 |   38.09524
 4.429 |   54.85714
 5.000 |   74.66667


In [36]:
# Gradient descent
alpha = 0.01
gradient = tf.reduce_mean(tf.multiply(tf.multiply(W, X) - Y, X))
descent = W - tf.multiply(alpha, gradient)
W.assign(descent)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1.0032725>

In [51]:
# Gradient descent code
tf.random.set_seed(0) # for reproducibility

x_data = [1., 2., 3., 4.]
y_data = [1., 3., 5., 7.]

# 정규분포를 따르는 random W [1]은 shape를 의미
# W = tf.Variable(tf.random.normal([1], -100., 100.))
# W에 아무거나 줘보자
W = tf.Variable([30.0])

for step in range(300):
  hypothesis = W * x_data
  cost = tf.reduce_mean(tf.square(hypothesis - y_data))

  alpha = 0.01
  gradient = tf.reduce_mean(tf.multiply(tf.multiply(W, x_data) - y_data, x_data))
  descent = W - tf.multiply(alpha, gradient)
  W.assign(descent)

  if step % 10 == 0:
    print('{:5} | {:10.4f} | {:10.6f}'.format(step, cost.numpy(), W.numpy()[0]))

    0 |  6021.0000 |  27.875000
   10 |  1266.3342 |  13.685345
   20 |   266.4390 |   7.178221
   30 |    56.1631 |   4.194168
   40 |    11.9426 |   2.825734
   50 |     2.6431 |   2.198195
   60 |     0.6875 |   1.910416
   70 |     0.2762 |   1.778446
   80 |     0.1897 |   1.717927
   90 |     0.1715 |   1.690174
  100 |     0.1677 |   1.677447
  110 |     0.1669 |   1.671610
  120 |     0.1667 |   1.668934
  130 |     0.1667 |   1.667706
  140 |     0.1667 |   1.667143
  150 |     0.1667 |   1.666885
  160 |     0.1667 |   1.666767
  170 |     0.1667 |   1.666713
  180 |     0.1667 |   1.666688
  190 |     0.1667 |   1.666676
  200 |     0.1667 |   1.666671
  210 |     0.1667 |   1.666669
  220 |     0.1667 |   1.666667
  230 |     0.1667 |   1.666667
  240 |     0.1667 |   1.666667
  250 |     0.1667 |   1.666667
  260 |     0.1667 |   1.666667
  270 |     0.1667 |   1.666667
  280 |     0.1667 |   1.666667
  290 |     0.1667 |   1.666667


In [54]:
# Gradient descent code2
tf.random.set_seed(0) # for reproducibility

x_data = [1., 2., 3., 4.]
y_data = [1., 3., 5., 7.]

# 정규분포를 따르는 random W [1]은 shape를 의미
# W = tf.Variable(tf.random.normal([1], -100., 100.))
# W에 아무거나 줘보자
W = tf.Variable([30.0])

alpha = 0.01

for step in range(300):
  with tf.GradientTape() as tape:
    hypothesis = W * x_data
    cost = tf.reduce_mean(tf.square(hypothesis - y_data))

  W_grad = tape.gradient(cost, [W])[0]
  W.assign_sub(alpha * W_grad)

  if step % 10 == 0:
    print('{:5} | {:10.4f} | {:10.6f}'.format(step, cost.numpy(), W.numpy()[0]))

    0 |  6021.0000 |  25.750000
   10 |   233.5314 |   6.408059
   20 |     9.2118 |   2.600126
   30 |     0.5173 |   1.850441
   40 |     0.1803 |   1.702847
   50 |     0.1672 |   1.673790
   60 |     0.1667 |   1.668069
   70 |     0.1667 |   1.666943
   80 |     0.1667 |   1.666721
   90 |     0.1667 |   1.666677
  100 |     0.1667 |   1.666669
  110 |     0.1667 |   1.666667
  120 |     0.1667 |   1.666667
  130 |     0.1667 |   1.666667
  140 |     0.1667 |   1.666667
  150 |     0.1667 |   1.666667
  160 |     0.1667 |   1.666667
  170 |     0.1667 |   1.666667
  180 |     0.1667 |   1.666667
  190 |     0.1667 |   1.666667
  200 |     0.1667 |   1.666667
  210 |     0.1667 |   1.666667
  220 |     0.1667 |   1.666667
  230 |     0.1667 |   1.666667
  240 |     0.1667 |   1.666667
  250 |     0.1667 |   1.666667
  260 |     0.1667 |   1.666667
  270 |     0.1667 |   1.666667
  280 |     0.1667 |   1.666667
  290 |     0.1667 |   1.666667
